# Diagnosis of COVID-19 Virus using IncpetionV3

### Import Packages

In [1]:
# import the necessary packages
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.optimizers import RMSprop
from sklearn.metrics import confusion_matrix
from imutils import paths
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import argparse
import cv2
import os
import pandas as pd
import cufflinks as cf
cf.go_offline()

### Set up the path to data and model

In [51]:
path_to_data = "/Users/abusnina/Documents/Training/COVID_19/dataset"
path_to_inceptionV3_model = "/Users/abusnina/Documents/Training/COVID_19/"

In [52]:
# initialize the initial learning rate, number of epochs to train for, and batch size
INIT_LR = 1e-3
EPOCHS = 100
BS = 10

### Load Data

In [53]:

imagePaths = list(paths.list_images(path_to_data))
data = []
labels = []
# loop over the image paths
for imagePath in imagePaths:
    # extract the class label from the filename
    label = imagePath.split(os.path.sep)[-2]
    # load the image, swap color channels, and resize it to be a fixed
    # 224x224 pixels while ignoring aspect ratio
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (224, 224))
    # update the data and labels lists, respectively
    data.append(image)
    labels.append(label)
# convert the data and labels to NumPy arrays while scaling the pixel
# intensities to the range [0, 1]
data = np.array(data) / 255.0
labels = np.array(labels)


[INFO] loading images...


### One-hot encoding and Generator

In [55]:
# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.20, stratify=labels, random_state=42)
# initialize the training data augmentation object
trainAug = ImageDataGenerator(rotation_range=15, 
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True, fill_mode="nearest")

### Transfer Learning based on InceptionV3

In [70]:
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [71]:
local_weights_file = path_to_inceptionV3_model + 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
pre_trained_model = InceptionV3(input_shape = (224, 224, 3),  include_top = False,  weights = None)
pre_trained_model.load_weights(local_weights_file)
# pre_trained_model.summary()
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


In [72]:
last_output = AveragePooling2D(pool_size=(4, 4))(last_output)
# Flatten the output layer to 1 dimension
x = Flatten()(last_output)
# Add a fully connected layer with 1,024 hidden units and ReLU activation
x = Dense(64, activation='relu')(x)
# Add a dropout rate of 0.2
x = Dropout(0.5)(x)                  
# Add a final sigmoid layer for classification
x = Dense  (2, activation='softmax')(x)  

model = Model( pre_trained_model.input, x) 
for layer in pre_trained_model.layers:
    layer.trainable = False

    #opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
#opt = RMSprop(lr=0.001) #, rho=0.9, epsilon=1e-08, decay=0.0)  

model.compile(optimizer = RMSprop(lr=0.0001, decay=0.0001 / 100), 
              loss = 'binary_crossentropy', 
              metrics = ['acc'])



In [73]:
 
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('acc')>1.0):
            print("\nReached 100% accuracy so cancelling training!")
            self.model.stop_training = True

callbacks = myCallback()           
# train the head of the network
print("[INFO] training head...")
H = model.fit_generator(trainAug.flow(trainX, trainY, batch_size=BS), 
                        steps_per_epoch=len(trainX) // BS,
                        validation_data=(testX, testY),
                        validation_steps=len(testX) // BS, epochs=EPOCHS,  callbacks=[callbacks])

[INFO] training head...
Epoch 1/100
15/15 [==============================] - 91s 6s/step - loss: 0.6220 - acc: 0.6443 - val_loss: 0.5715 - val_acc: 0.8750
Epoch 2/100
15/15 [==============================] - 91s 6s/step - loss: 0.5647 - acc: 0.7315 - val_loss: 0.5384 - val_acc: 0.7000
Epoch 3/100
15/15 [==============================] - 92s 6s/step - loss: 0.4898 - acc: 0.8456 - val_loss: 0.5473 - val_acc: 0.5750
Epoch 4/100
15/15 [==============================] - 92s 6s/step - loss: 0.4471 - acc: 0.8400 - val_loss: 0.4081 - val_acc: 0.8250
Epoch 5/100
15/15 [==============================] - 92s 6s/step - loss: 0.4449 - acc: 0.8581 - val_loss: 0.3556 - val_acc: 0.8000
Epoch 6/100
15/15 [==============================] - 98s 7s/step - loss: 0.3873 - acc: 0.9060 - val_loss: 0.3584 - val_acc: 0.7750
Epoch 7/100
15/15 [==============================] - 97s 6s/step - loss: 0.3524 - acc: 0.9262 - val_loss: 0.3106 - val_acc: 0.8500
Epoch 8/100
15/15 [==============================] - 92s 6s

In [74]:
# make predictions on the testing set
predIdxs = model.predict(testX, batch_size=BS)
# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)
# show a nicely formatted classification report
print(classification_report(testY.argmax(axis=1), predIdxs, target_names=lb.classes_))

[INFO] evaluating network...
              precision    recall  f1-score   support

       covid       0.95      1.00      0.98        20
      normal       1.00      0.95      0.97        20

    accuracy                           0.97        40
   macro avg       0.98      0.97      0.97        40
weighted avg       0.98      0.97      0.97        40



In [20]:
df = pd.DataFrame(H)
df.to_csv("Training_history.csv", index=False)

In [76]:
model.save('InceptionV3_COVID_19.h5')

In [17]:
Path_to_model = "InceptionV3_COVID_19.h5"


In [26]:
fig = df.iplot(asFigure=True)
fig.show()

In [3]:
df= pd.read_csv("Training_history.csv")

In [7]:
fig = df[["loss","val_loss"]].iplot(asFigure=True, kind="histogram", bins=100)
fig.show()